<a href="https://colab.research.google.com/github/ucfilho/optim_raianars/blob/master/ABCOptim_jun_16_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
"""ABCOptim_jun_10_2019.ipynb
Automatically generated by Colaboratory.
Original file is located at
    https://colab.research.google.com/drive/1KxLGSILwQZQySSynZeoDYtyG4TLz_IRh
"""

import numpy as np
from random import randint
#import Function
#import Enxame

''' 
FOBJ gera vetorialmente os valores com a funcao objetivo escalar
em outras palavras gera para populacao o valor da funcao 
a ser otimizada
'''


def Fun(x):
    
    summ=0
    for i in range(len(x)):
        new=x[i]*np.sin((abs(x[i]))**0.5)
        summ=summ+new
#        print(summ)
    return (418.9829*len(x)-summ)  

def FOBJ(X,Fun):
    rows = X.shape[0]
    fobj=np.zeros(rows)
    for i in range(rows):
        fobj[i]=Fun(X[i,])
    return fobj

'''Enxame retorna a populacao aleatoria com todos valores entre MIN e MAX'''
def Enxame(PAR,NPAR,MAX,MIN):
    x=np.zeros((NPAR, len(MAX)))
    for j in range(len(MAX)):
        for i in range(NPAR):
            x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x

''' 
Fitness gera vetorialmente os valores com a funcao fitness
equivale a quantidade de nectar da fonte
'''
def FIT(X,Fun):
    rows = X.shape[0]
    fit=np.zeros(rows)
    fit=np.copy(FOBJ(X,Fun))
    for i in range(rows):
        if(fit[i]>=0):
            fit[i]=1/(fit[i]+1)
        else:
            fit[i]=1+abs(fit[i])
    return fit

'''
probablilidade usada para roleta q atualizacao das onlookers 
semelhante ao genetico
'''

def PROB(x,Fun):
    rows=x.shape[0]
    fit=np.zeros(rows)
    prob=np.zeros(rows)
    fit=FIT(x,Fun)
    fit_ref=max(fit)
    
    for i in range(rows):
        prob[i] = 0.9*fit[i]/fit_ref+0.1

    return prob

'''
Obtem a melhor escolha greedy search
'''
def GetBest(x,xbest,Fun):
    ycal=FOBJ(x,Fun)
    best=np.argmin(ycal)
    yref=Fun(xbest)
    if(yref<ycal[best]):
        GBEST=xbest
    else:
        GBEST=x[best,]
    return GBEST

'''
employed bee phase
'''
def EmployedBee(xo,x,MAX,MIN,trial,Fun) :
    NPAR = xo.shape[0]
    PAR= xo.shape[1]
    V=np.zeros((NPAR, len(MAX)))

    for j in range(PAR):
        for i in range(NPAR):
            fi=np.random.uniform(low=-1.0, high=1.0, size=None)
            rd=randint(0, (NPAR-1))
            V[i,j]=xo[i,j]+(xo[i,j] -x[rd,j])*fi # candidata a solucao
            if (V[i,j]>MAX[j]):
              V[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
            if (V[i,j]<MIN[j]):
              V[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
              
    YCAL=FOBJ(xo,Fun)
    YV=FOBJ(V,Fun)
    for i in range(NPAR):
        if(YV[i]<YCAL[i]):
          trial[i]=0
          xo[i,]=np.copy(V[i,]) #MODIFICADO JUN 12
        else:
          trial[i]=trial[i]+1
            

    return xo,trial  #MODIFICADO JUN 12

'''
onlooker bee phase
'''
def OnlookerBee(xo,x,trial,MAX,MIN,Fun): #OnlookerBee(xo,x_Employed)
    rows = xo.shape[0]
    cols = xo.shape[1]
    prob=PROB(x,Fun)
    prob_ref=max(prob)
    #best=np.argmax(prob)
    
    for i in range(rows):
      prob_ref=np.random.uniform(low=0, high=1.0, size=None)
      rd=randint(0, (rows-1))
      if(prob[i] > prob_ref):  
        fi=np.random.uniform(low=-1.0, high=1.0, size=None)
        Xmi=xo[i,]+fi*(xo[i,]-x[rd,])
        xo[i,]=np.copy(Xmi)
        for j in range(cols):
          if (xo[i,j]>MAX[j]):
            xo[i,j]=np.copy(MAX[j]) # restringe a busca para o intervalo
          if (xo[i,j]<MIN[j]):
            xo[i,j]=np.copy(MIN[j]) # restringe a busca para o intervalo
    
    YCAL=FOBJ(xo,Fun)
    YV=FOBJ(x,Fun)
        
    for i in range(rows):
      if(YV[i]<YCAL[i]):
        trial[i]=0
        xo[i,]=np.copy(x[i,])
      else:
        trial[i]=trial[i]+1
    return xo,trial

'''
scout bee phase
'''
def ScoutBee(trial,x,ntrail,MAX,MIN):
#def ScoutBee(x,trial,MAX,MIN):
    PAR = x.shape[0]
    NPAR = x.shape[1]
    for i in range(PAR):
      if(trial[i]>= ntrail):
        trial[i]=0
        for j in range(NPAR):
          x[i,j]=MIN[j]+(MAX[j]-MIN[j])*np.random.random()
    return x,trial

'''
Metodo que alterna e coordena o uso employers,onlooker, scout bees
e greedy search
'''
def ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN,Fun):

    trial=np.zeros(NPAR) # inicializa contador p abandonar fonte de alimento
    xbest=Enxame(PAR,1,MAX,MIN)[0,] #inicializa xbest
    xo=Enxame(PAR,NPAR,MAX,MIN) # inicializa employed bee
    xbest=GetBest(xo,xbest,Fun)
    
    for i in range(ITE):
      x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para employed bee
      xo,trial=EmployedBee(xo,x,MAX,MIN,trial,Fun)
      xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
      x=Enxame(PAR,NPAR,MAX,MIN)  # comparativo para OnlookerBee
      xo,trial=OnlookerBee(xo,x,trial,MAX,MIN,Fun)
      xbest=GetBest(xo,xbest,Fun) 
      #xbest=GetBest(xo,xbest)
      # x=Enxame(PAR,NPAR,MAX,MIN) # comparativo para ScoutBee
      xo,trial=ScoutBee(trial,x,ntrail,MAX,MIN) #MODIFICADO JUN 12
      xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
    x=np.copy(xo)
    xbest=GetBest(xo,xbest,Fun) #MODIFICADO JUN 12
    
    return xbest, x,trial

#'''
    
dim=3
#Fun=Function.Schwefel
MAX=np.repeat(500,dim)
MIN=np.repeat(-500,dim)

NPAR=175 #PARTICULAS (fontes de comida)
ITE=300 #ITERACOES
PAR=len(MAX)
ntrail=75 #numero de buscas ate abandonar uma fonte de alimento

for i in range(20):
    print("resolucao",i+1," ")
    xbest, x,trial =ABCOPtim(ITE,PAR,ntrail,NPAR,MAX,MIN,Fun)
    print("vetor",xbest,"funcao", Fun(xbest),"\n")

#print(trial)
#'''

resolucao 1  
vetor [412.67081252 412.85391317 421.59762564] funcao 16.936203759629507 

resolucao 2  
vetor [404.30350839 431.11838251 428.64024749] funcao 54.738280646403155 

resolucao 3  
vetor [418.77313331 429.33168362 411.26707978] funcao 21.20627395718725 

resolucao 4  
vetor [419.70537396 420.20466318 410.27555188] funcao 14.560751909774353 

resolucao 5  
vetor [440.76213423 419.60381257 439.08016679] funcao 90.0974138325339 

resolucao 6  
vetor [423.87184605 418.94773891 426.68054552] funcao 5.698145467712493 

resolucao 7  
vetor [423.33616808 421.33782945 415.52288788] funcao 4.453729977683906 

resolucao 8  
vetor [418.51077527 429.11516652 427.28088559] funcao 14.163670252841484 

resolucao 9  
vetor [423.01961822 406.67029488 436.27321721] funcao 55.30640552965497 

resolucao 10  
vetor [422.21936089 414.10492684 436.31020423] funcao 35.63904882972588 

resolucao 11  
vetor [426.23565258 417.12398831 426.01093863] funcao 8.573937253282566 

resolucao 12  
vetor [418.9

In [2]:
FOBJ(x,Fun)

array([1144.85106744, 1266.5144214 ,  454.02779391, 1426.94618463,
       2046.3769815 , 1162.69998469, 1275.74507079, 1395.36309903,
       1326.10962884, 1634.94148812,  956.54191406,  980.06424676,
        701.99346842, 1179.55076524, 1353.50557561, 1500.90908786,
       1625.27785035,  823.02441808, 1685.81729607, 1237.83182339,
       1046.20963745, 1544.92138826, 1788.76631576, 1157.35124803,
       1627.69004403,  694.10344956,  969.61252614, 1297.34379272,
       1758.57257731, 1147.9379962 , 1755.71638313, 1389.02646064,
       1023.57061515, 1397.07846758, 1071.77720921, 1343.67295529,
       1388.71156902,  759.44282374,  966.99837409, 1448.7625588 ,
        673.85174313, 1668.50455622,  641.61684897, 1342.47710066,
        619.86410212, 1363.86367632, 1046.32895777, 1377.16044681,
       1095.3627505 , 1753.36789529,  778.17586354, 1154.79521979,
       1381.18612663, 1141.79432366, 1874.30823515, 1091.64328682,
       1058.28931764, 1386.40842419,  971.68681208,  823.77200

In [3]:
min(FOBJ(x,Fun))

362.86046446109526

In [4]:
Fun(xbest)

8.766393896246427